In [6]:
import pandas as pd

# Load your merged dataset
df = pd.read_csv("btc_all_time.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Resample to weekly close prices
weekly = df['Close'].resample('W-MON').last().dropna()
weekly_df = pd.DataFrame({'Close': weekly})
weekly_df['Weekly_Change'] = weekly_df['Close'].pct_change() * 100
weekly_df['Week'] = weekly_df.index

# Define historical cycle peaks
peak_windows = {
    "2013": ("2013-11-18", "2013-12-09"),
    "2017": ("2017-12-04", "2017-12-25"),
    "2021": ("2021-11-01", "2021-11-22"),
}

# Create DataFrame of past peak weeks
peak_data = []
for cycle, (start, end) in peak_windows.items():
    period = weekly_df.loc[start:end]
    for _, row in period.iterrows():
        peak_data.append({
            'Cycle': cycle,
            'Week': row['Week'],
            'Close': row['Close'],
            'Weekly_Change': row['Weekly_Change']
        })

df_peaks = pd.DataFrame(peak_data)
df_peaks.reset_index(drop=True, inplace=True)
df_peaks


,Cycle,Week,Close,Weekly_Change
0,2013,2013-11-18,785.4,116.363636
1,2013,2013-11-25,830.0,5.678635
2,2013,2013-12-02,1096.6,32.120482
3,2013,2013-12-09,919.0,-16.195513
4,2017,2017-12-04,11623.9,19.425266
5,2017,2017-12-11,16732.5,43.949105
6,2017,2017-12-18,18972.3,13.385926
7,2017,2017-12-25,13833.5,-27.085804
8,2021,2021-11-01,60915.3,-3.411768
9,2021,2021-11-08,67527.9,10.855401


In [11]:
# Get the most recent 4 weeks from the full dataset
recent_weeks = df.copy()
recent_weeks['Week'] = recent_weeks.index.to_period('W').to_timestamp()
weekly = recent_weeks.groupby('Week').agg({'Close': 'last'}).reset_index()
weekly['Weekly_Change'] = weekly['Close'].pct_change() * 100

# Get last 4 weeks
latest_trend = weekly.tail(4).copy()
latest_trend.reset_index(drop=True, inplace=True)

# Show it
streak = latest_trend['Weekly_Change'].values
print("📊 Latest 4-Week BTC Trend:")
print(latest_trend)

# Compare it to historical patterns
def is_peak_like(pattern):
    return (
        len(pattern) == 4 and
        pattern[0] > 10 and
        pattern[1] > 10 and
        pattern[2] < 10 and
        pattern[3] < 0
    )

if is_peak_like(streak):
    print("🚨 Warning: This resembles a historical cycle peak!")
else:
    print("✅ Current trend does not strongly resemble a past peak.")


📊 Latest 4-Week BTC Trend:
        Week      Close  Weekly_Change
0 2025-06-23  108356.93       7.322481
1 2025-06-30  109203.84       0.781593
2 2025-07-07  119086.64       9.049865
3 2025-07-14  120354.94       1.065023
✅ Current trend does not strongly resemble a past peak.
